In [201]:
# Merge multi Hans Hval
hans = Contributor.objects.search("Hval").exclude(display_name__contains='Olav')
from utils.merge_model_objects import merge_instances
merge_instances(*list(hans))
hans

<ContributorQuerySet [<Contributor: Hans Dalane-Hval>]>

In [172]:
import re
def re_find(expression, haystack):
    try:
        return re.search(expression, haystack).groups()[0]
    except AttributeError:
        return ''

def get_initials_from_file_name(filename):
    return re_find(r'-(?P<initials>[A-Z]+)((-NR)?-?\d+)?(-*\d+-of-\d+)?(_\w{7})?\.\w+$', filename)

In [173]:
def active_photographers(when=None):
    photographer = Position.objects.get(title='fotograf')
    stints = Stint.objects.filter(position=photographer)
    if when:
        stints = stints.active(when)
    contributors = Contributor.objects.filter(pk__in=stints.values_list('contributor', flat=True))
    return contributors

In [174]:
import logging
logger = logging.getLogger()
logger.setLevel(logging.ERROR)
from utils.disconnect_signals import disconnect_signals

In [ ]:
# Byline images
ImageFile.objects.filter(source_file__startswith='byline-photo').update(category=ImageFile.PROFILE)

In [203]:
# Update photographer initials
def get_initials(name):
    name = name.translate(str.maketrans('ÆØÅ', 'AOA'))
    return ''.join(re.findall(r'[A-Z]', name))[:5]

def is_photographer(cn):
    if cn.stint_set.filter(position__title='fotograf').count():
        return True
    if cn.byline_set.filter(title='foto').count():
        return True
    if cn.imagefile_set.count():
        return True
    else:
        return False
    
    
for cn in Contributor.objects.all().order_by('display_name'):
    if is_photographer(cn):
        initials = get_initials(str(cn))
        if cn.initials != initials:
            cn.initials = initials
            cn.save() 
        print(f'{str(cn.initials):<6} {name:<30} {cn.user or "-"}')
    elif cn.initials:
        cn.initials = ''
        cn.save() 
    name = str(cn)

NN     NN                             -
NN     NN                             -
AN     Ådne Hindenes                  -
AKV    Aksel Hagen                    -
AM     Aleksander Martin Gjøsæter     -
AQG    Alexander Løtvedt              -
AS     Alf Jensen                     -
ABP    Amalie Rossland Christiansen   -
AOB    Amanda Borch Pacheco           -
AO     Amina Storm                    -
AA     Amund Trellevik                -
AAH    Anders Aasbø                   -
AFL    Anders C. Heli                 -
ANK    Anders Myklebust               -
AR     Anders R. Christensen          -
AV     Anders Sundstøl Bjørkheim      -
AN     Andreas Nebdal                 -
AC     Anette Remme                   -
AW     Anna Vibeke Lorentzen          -
ABV    Anna Young                     -
ALG    Anne Kathrine Wexels Wold      -
AO     Anne Nordheim                  -
A      Arild Underdal                 -
AH     Asbjørn Slettemark             -
ABE    Åse Michaelsen                 -


OHF    Ole Henrik Ellestad            -
OS     Olga S. Meh                    -
OG     Øystein Engedal                -
OV     Øystein Vaglid                 -
OA     Øyunn Syrstad Høydal           -
OGSD   Øyvind Gallefoss               -
OL     Øyvind Kind Robertsen          -
PA     Øyvind Sundstrøm               -
PSS    Partynorsk                     -
PAW    Patrick Egersborg              -
PPB    Paul Grini                     -
PF     Petter Brønstad                -
PH     Petter Haas Brubakk            -
PAJ    Petter Risøe                   -
P      Poteten                        -
RT     Ragnar Lie                     -
RP     Rahma Søvik                    -
RE     Riboflav                       -
RBF    Rita Helgesen                  roarblasmo
RBBF   Roar Blåsmo-Falnes             -
RRJ    Robin Håset Drager             -
RO     Rune Olav Horstad              -
SB     Samselius                      -
SJ     Sander Brink                   -
SN     Sara Moss               

In [191]:
# Illustrations
kvammen = Contributor.objects.search('Anders Nordmo Kvammen')[0]
kvammen.initials = 'ANK'
kvammen.save()
kvammen_illus= ImageFile.objects.filter(source_file__contains='-ANK.jpg')
kvammen_illus.update(category=ImageFile.ILLUSTRATION, contributor=kvammen)

hovland = Contributor.objects.search('Øivind Hovland')[0]
hovland_illus = ImageFile.objects.filter(description__contains='vind Hovland')
hovland_illus.update(category=ImageFile.ILLUSTRATION, contributor=hovland)

159

In [190]:
# fix some contributor's initials
sjur = Contributor.objects.search('Sjur Stølen').first()
sjur.initials='SGS'
sjur.save()
ingrid = Contributor.objects.search('Ingrid Dalen Rogstad').first()
ingrid.initials = 'IDR'
ingrid.save()
vilde = Contributor.objects.search('Vilde Borse').first()
vilde.initials = 'VIB'
vilde.save()
aaah = Contributor.objects.search('Anders Aavik Hagen')[0]
ImageFile.objects.filter(contributor=None, source_file__contains='AAAH').update(contributor=aaah)

0

In [192]:
# Initials lookup dictionary
lookup = {cn.initials: cn for cn in Contributor.objects.exclude(initials='') if 1 < len(cn.initials) < 4}
lookup['HHH'] = lookup['HDH']
lookup['TWM'] = lookup['TM']
lookup['AAGS'] = lookup['AS']
lookup['AHA'] = lookup['AH']
lookup['PP'] = lookup['PPB']

In [193]:
def guess_photo_contributor(image_file):
    """Use various methods to determine probable contributor of image""" 
    name, contributor = None, None
    if image_file.contributor:
        contributor = image_file.contributor
        logger.debug('already assigned')
        return contributor
    if image_file.exif:
        # exif data lookup
        exif = image_file.exif
        names = [exif.artist, re_find(r'[fF]oto:(.*)^', exif.description)]
        name = sorted(names, key=len)[-1]
    if name and not contributor:
        try:
            contributor = Contributor.objects.get(display_name=name)
            logger.debug(f'found name {name}')
        except Contributor.DoesNotExist:
            matches = Contributor.objects.search(name)
            if matches.count() == 1:
                contributor = matches[0]
                logger.debug(f'fuzzy search for name {name} -> {contributor}')
    initials = get_initials_from_file_name(image_file.original.name)
    if initials and not contributor:
        try:
            contributor = active_photographers(when=image_file.created).get(initials=initials)
            logger.debug(f'found initials {initials}')
        except Contributor.DoesNotExist:
            # try from lookup list
            contributor = lookup.get(initials) 
                
    if not contributor:
        # look at bylines for stories
        sim = image_file.storyimage_set.first()
        if sim:
            photo_byline = sim.parent_story.byline_set.filter(credit='photo')
            if photo_byline.count() == 1:
                contributor = photo_byline.first().contributor
                logger.debug(f'found photo byline {photo_byline.first()}')
    
    if not contributor and (name or initials):
        logger.warn(f'{image_file} did not find {name} / {initials}')
    return contributor

def assign_contributor(qs):
    disconnect_signals()
    success = []
    fail = []
    for im in qs:
        cn = guess_photo_contributor(im)
        if cn:
            im.contributor = cn
            im.category = ImageFile.PHOTO
            im.save()
            success.append(f'{im.pk:<5} {str(cn):<40} {im}')
        else:
            fail.append(str(im))
    print('OK:')
    print('\n'.join(success))
    print('\nFAIL:')
    print(len(fail))
        
qs = ImageFile.objects.filter(contributor=None)
print('unassigned', qs.count())
assign_contributor(qs)
qs = ImageFile.objects.filter(contributor=None)
print('unassigned', qs.count())

unassigned 10534
OK:
39282 Vilde Borse                              31-MAG-handikap-VIB-16.jpg
39280 Vilde Borse                              31-MAG-handikap-VIB-15.jpg
39277 Vilde Borse                              31-MAG-handikap-VIB-13.jpg
39276 Vilde Borse                              31-MAG-handikap-VIB-12.jpg
39275 Vilde Borse                              31-MAG-handikap-VIB-11.jpg
39274 Vilde Borse                              31-MAG-handikap-VIB-10.jpg
39272 Vilde Borse                              31-MAG-handikap-VIB-8.jpg
39270 Vilde Borse                              31-MAG-handikap-VIB-6.jpg
39268 Vilde Borse                              31-MAG-handikap-VIB-4.jpg
38324 Vilde Borse                              26-KUN-Kunst-pa-HiOA-VIB15.jpg
38322 Vilde Borse                              26-KUN-Kunst-pa-HiOA-VIB13.jpg
38321 Vilde Borse                              26-KUN-Kunst-pa-HiOA-VIB12.jpg
38319 Vilde Borse                              26-KUN-Kunst-pa-HiOA-VIB10.jpg
3831

In [195]:
fix = {
    " Prof. Ingrid Os": "Ingrid Os",
    ". Translated By Michael Brøndbo": "Michael Brøndbo",
    "Adrian Nielsen": "Adrian Nielsen",
    "Adrian Simen Holm": "Adrian Simen Holm",
    "Aksel Schreiner Brakestad": "Aksel Schreiner Brakestad",
    "Aleksander Myklebust": "Aleksander Myklebust",
    "Alf Grønli Simensen": "Alf Simensen",
    "Amanda O. Berg": "Amanda O. Berg",
    "Anders Aa. Hagen": "Anders Aavik Hagen",
    "Anders Imenes": "Anders Gravir Imenes",
    "Anders Jupskås": "Anders Ravik Jupskås",
    "Anders Nordmo Kvammen": "Anders Nordmo Kvammen",
    "Anders R. Christensen": "Anders R. Christensen",
    "Anders Schiøtz Worren": "Anders Schiøtz Worren",
    "Anders Sundstøl Bjørkheim": "Anders Sundstøl Bjørkheim",
    "Anders Veberg": "Anders Veberg",
    "Andreas Slørdahl": "Andreas Slørdahl",
    "Andreas Øien Stensland": "Andreas Øien Stensland",
    "André Larsen Avelin": "André Larsen Avelin",
    "Angelique Culvin": "Angelique Culvin",
    "Anja Kristine Salo": "Anja Kristine Salo",
    "Anne B. Viken": "Anne Bergitte Viken",
    "Anne Fougner Helseth": "Anne Fougner Helseth",
    "Anne Leifsdatter Grønlund": "Anne Leifsdatter Grønlund",
    "Are W. Sandvik": "Are Westerink Sandvik",
    "Aslak Johannessen": "Aslak Johannesen",
    "Aud V. Tønnessen": "Aud V. Tønnessen",
    "Aud Valborg Tønnesen": "Aud Valborg Tønnesen",
    "August Tekrø": "August Tekrø",
    "Aurora Sæverud": "Aurora Sæverud",
    "Axel Geard Nygaard": "Axel Geard Nygaard",
    "Axel M. K. Hærland": "Axel Munthe-Kaas Hærland",
    "Bendik Baksaas": "Bendik Baksaas",
    "Benedicte Tandsæther-Andersen": "Benedicte Tandsæther-Andersen",
    "Benjamin Endré Larsen": "Benjamin Endré Larsen",
    "Benjamin Oliver": "Benjamin Edward Oliver",
    "Birgit Skarstein": "Birgit Røkkum Skarstein",
    "Birte Nystad Magnussen": "Birte Nystad Magnussen",
    "Bjarne Henning Kvåle": "Bjarne Henning Kvaale",
    "Bjørnar K. Bekkevard": "Bjørnar Konglevoll Bekkevard",
    "Bjørn–Egil Mikalsen": "Bjørn-Egil Mikalsen",
    "Brage Herlofsen": "Brage Herlofsen",
    "Brian Olguin": "Brian Olguin",
    "Brita Søreng": "Brita Melissa Botnen Søreng",
    "Camilla Brønnich Eikeland": "Camilla Brønnich Eikeland",
    "Carina I. Hunshamar": "Carina Isabelle Hunshamar",
    "Cecilie Einarson Pérez": "Cecilie Einarson Pérez",
    "Christian Lycke": "Christian Lyche",
    "Christina Bock Bjørn-Hansen": "Christina Bock Bjørn–Hansen",
    "Christina Bock": "Christina Bock",
    "Christine Skogen Nyhagen": "Christine Skogen Nyhagen",
    "Daniel Heggelid-Rugaas": "Daniel Heggelid-Rugaas",
    "Eirik B. Traavik": "Eirik B. Traavik",
    "Eirik Billingsø Elvevold": "Eirik Billingsø Elvevold",
    "Elisabeth Rørbakk Jakobsen": "Elisabeth Rørbakk Jakobsen",
    "Elisabeth Torkildsen": "Elisabeth Torkildsen",
    "Ellen Munden Paalgard": "Ellen Munden Paalgard",
    "Ellen Paalgaard": "Ellen Munden Paalgaard",
    "Ellinor Bente Dalbye": "Ellinor Bent Dalbye",
    "Embret T. Rognerød": "Embret T. Rognerød",
    "Emilie Solberg": "Emilie Solberg",
    "Erika Kristine Ribu": "Erika Kristine Ribu",
    "Erling Hillestad": "Erling G. Hillestad",
    "Eva–Kristin Urestad Pedersen": "Eva-Kristin Urestad Pedersen",
    "Evelyn Pecori": "Evelyn Pecori",
    "Foto Angélique Culvin": "Angélique Culvin",
    "Frederick Alexander Reiersen": "Frederick Alexander Reiersen",
    "Gaute M. Kornberg": "Gaute M. Kornberg",
    "Grethe Kielland-Jenssen": "Grethe Kielland-Jenssen",
    "Gunhild Oland Nedrelid": "Gunhild Oland Nedrelid",
    "Haakon Jamtli Kristiansen": "Haakon Jamtli Kristiansen",
    "Hallvard Østtveit Barbogen": "Hallvard Østtveit Barbogen",
    "Halvor Finess Tretvoll": "Halvor Finess Tretvoll",
    "Hanna Kristin Hjardar": "Hanna Kristin Hjardar",
    "Hanna Magnussen Krogvold": "Hanna Magnussen Krogvold",
    "Hanna Skotheim": "Hanna Skotheim",
    "Hanne Hvattum": "Hanne O. Hvattum",
    "Hanne Marie Lenth Solbø": "Hanne Marie Lenth Solbø",
    "Hans Dalane-Hval": "Hans Dalane-Hval",
    "Hans J. Skjong": "Hans Joseph Skjong",
    "Harald Fagerheim Bugge": "Harald Fagerheim Bugge",
    "Hedvig Mollestad Thomassen": "Hedvig Mollestad Thomassen",
    "Heidi T. Harto": "Heidi Thaulow Harto",
    "Helena Nielsen": "Helena Nielsen",
    "Helene Kristensen": "Helene Kristiansen",
    "Hilde Berteig Rustan": "Hilde Berteig Rustan",
    "Håkon Borgos": "Håkon Borgos",
    "Håkon Frede Foss": "Håkon Frede Foss",
    "Ida Elizabeth Torp Halvorsen": "Ida Elizabeth Torp Halvorsen",
    "Ida Lyngstad Wernø": "Ida Lyngstad Wernø",
    "Ida Madsen Hestman": "Ida Madsen Hestman",
    "Igor Nilsen": "Igor R. Nilsen",
    "Ine Eriksen Søreide": "Ine Eriksen Søreide",
    "Ingeborg Huse Amundsen": "Ingeborg Huse Amundsen",
    "Ingeborg Marie Østby Laukvik (Leder": "Ingeborg Marie Østby Laukvik",
    "Ingeborg Skjelkvåle Ligaarden": "Ingeborg Skjelkvåle Ligaarden",
    "Ingjerd Sørlie": "Ingjerd Sørli",
    "Ingri Bergo": "Ingri G. Bergo",
    "Ingrid Aune": "Ingrid Aune",
    "Ingrid Eidsheim Daae": "Ingrid E. Daae",
    "Ingrid Elise Gipling": "Ingrid Gipling",
    "Ingrid H. F. Brubaker": "Ingrid Flognfeldt Brubaker",
    "Ingrid K. Fredriksen": "Ingrid Kvamme Fredriksen",
    "Ingrid N. Ekeberg": "Ingrid N. Ekeberg",
    "Ingrid Stranger–Thorsen": "Ingrid Stranger-Thorsen", 
    "Ingvild S. Reymert": "Ingvild S. Reymert",
    "Iselin Rud–Goksøyr": "Iselin Rud-Goksøyr",
    "Iselin Shaw of Tordarroch": "Iselin Shaw of Tordarroch",
    "Iván Kverme": "Ivàn Kverme",
    "Jamal Mohsin Masterstudent": "Jamal Mohsin",
    "Jan Thomas Holmlund": "Jan Thomas Holmlund",
    "Janni Kalafatis": "Janni Frederiksen Kalafatis",
    "Jens Å. R. Haga": "Jens Ådne R. Haga",
    "Joakim P. Berg": "Joakim P. Berg",
    "Julianne B. Mossing": "Julianne B. Mossing",
    "Julie Kalager": "Julie Kalager",
    "Julie Nordby Egeland": "Julie Nordby Egeland",
    "Julie Paus-Knudsen": "Julie Sørlie Paus-Knudsen",
    "June Torgersen": "June Marie Torgersen",
    "Jørgen Brynhildsvoll": "Jørgen Brynhildsvoll",
    "Jørn Wiche Pedersen": "Jørn Wichne Pedersen",
    "Kaj M. Duesund": "Kaj M. Duesund",
    "Kamilla Simonnes": "Kamilla Simonnes",
    "Karl Kristian Rådahl Kirchhoff": "Karl Kristian Rådahl Kirchhoff",
    "Katarina A. Rodina": "Katarina A. Rodina",
    "Katrine Elida Aaland": "Katrine Elida Aaland",
    "Ketil Blom": "Ketil Blom",
    "Kim Kantardjiev": "Kim Orlin Kantardjiev",
    "Kim Michael G. G. Thon": "Kim Michael G. G. Thon",
    "Kristian Ridder–Nielsen": "Kristian Ridder-Nilsen",
    "Kristin Jonassen Nordby": "Kristin Nordby",
    "Kristin Pettersen": "Kristin Pettersen",
    "Kristina Klakegg": "Kristina Klakegg",
    "Kristina Kvammen": "Kristina Elisabet Kvammen",
    "Kristoffer Hatteland Endresen": "Kristoffer Hatteland Endresen",
    "Lars Eng Førde": "Lars Engh Førde",
    "Lars-Christian U. Talseth": "Lars-Christian U. Talseth",
    "Leif Martin Kirknes": "Leif Martin Kirknes",
    "Lene C. Westgaard": "Lene C. Westgaard",
    "Lene Christensen": "Lene Christensen",
    "Lilly Susanne Stensland": "Susanne Stensland",
    "Line Ørnes Søndergaard": "Line Ørnes Søndergaard",
    "Lise Falck": "Lise Falch",
    "Lise Marie Danielsen": "Lise Marie Danielsen",
    "Liv Rønnaug Lilleåsen": "Liv Rønnaug Bjerke Lilleåsen",
    "Liv-Kristin Rød Korssjøen": "Liv-Kristin Rød Korssjøen",
    "Magnus Ekeland": "Magnus Ekeland",
    "Magnus K. Worren": "Magnus K. Worren",
    "Malene Hoftun Jæger": "Malene Hoftun Jæger",
    "Maren Blaalid": "Maren Blaalid",
    "Mari Helén Varøy Mari Mamre": "Mari Mamre",
    "Mari K. S. Gjelstad": "Mari K. S. Gjelstad",
    "Mari Lund Haanshus": "Mari Lund Haanshus",
    "Maria Reinertsen": "Maria Reinertsen",
    "Maria T. Pettrém": "Maria T. Pettrém",
    "Maria Terese Kittilsen": "Maria Terese Kittilsen",
    "Maria Terray Brantenberg": "Maria Terray Brantenberg",
    "Marianne Granheim Trøyflat": "Marianne Granheim Trøyflat",
    "Marianne Høva Rustberggard": "Marianne Høva Rustberggard",
    "Marianne Johansen": "Maria Mår Johansen",
    "Marit Aasmo Albrigtsen": "Marit Aasmoe Albrigtsen",
    "Marit Austvik Letnes": "Marit Austvik Letnes",
    "Marit M. Simonsen": "Marit M. Simonsen",
    "Marit Sunnanå Aalrust": "Marit Sunnanå Aalrust",
    "Marius Nergård Pettersen": "Marius N. Pettersen",
    "Marte B. L. Torskenæs": "Marte Lia Torskenæs",
    "Marte Engehaugen Bøhm": "Marte Engehaugen Bøhm",
    "Marthe Amanda Vannebo": "Marthe Amanda Vannebo",
    "Martin Anders Dahl": "Martin Anders Dahl",
    "Mats Kirkebirkeland": "Mats Kirkebirkeland",
    "Mats Sivertsen": "Mats Sivertsen",
    "Matthis Kleeb Solheim": "Matthis Kleeb Solheim",
    "Melike Leblebicioğlu": "Melike Leblebicioğlu",
    "Mirjam K. S. Folkvord": "Mirjam Sorge Folkvord",
    "Morten Oftedal Schwencke": "Morten Schwencke",
    "Nicolay Woldsdal": "Nicolay Woldsdal",
    "Nora Nygaard": "Nora Nygaard",
    "Ola Magnusson Rydje": "Ola Magnussen Rydje",
    "Patrick da Silva Sæther": "Patrick da Silva Sæther",
    "Paul Anthony Wilson": "Paul Anthony Wilson",
    "Paul Patrick Børhaug": "Paul Patrick Børhaug",
    "Peder Stabell": "Peder D. Stabell",
    "Per Anders Torvik Langerød": "Per Anders Langerød",
    "Petter Drange Brønstad": "Petter Brønstad",
    "Petter Risøe": "Petter Risøe",
    "Philip André Johannesborg": "Philip André Johannesborg",
    "Pia Sandved Berg": "Pia Sandved Berg",
    "Pål Vegard R. Hagesæther": "Pål Vegard Rameckers Hagesæther",
    "Rachel Elisabeth Antonsen": "Rachel Elisabeth Antonsen",
    "Ragnhild Eline Hoen": "Ragnhild Eline Hoen",
    "Regine Stokstad": "Regine Stokstad",
    "Robin Røkke Johansen": "Robin Røkke Johansen",
    "Rolv Christian Topdahl": "Rolv Christian Topdahl",
    "Runa Fjellanger": "Runa Fjellanger",
    "Runar B. Mæland": "Runar Bjørkvik Mæland", 
    "Sebastian Dahl": "Sébastian Dahl",
    "Sebastian Wemmerløv": "Sebastian Wemmerløv",
    "Signe Bakke Johannessen": "Signe Bakke Johannessen",
    "Sigurd Arnekleiv Bækkelund": "Sigurd Arnekleiv Bækkelund",
    "Sigurd K Berg": "Sigurd K. Berg",
    "Sigurd Oland Nedrelid": "Sigurd Oland Nedrelid",
    "Siri Øverland Eriksen": "Siri Øverland Eriksen",
    "Sjur Stølen": "Sjur Stølen",
    "Solveig Figenschou": "Solveig Figenschou",
    "Solveig Nygaard Langvad": "Solveig N. Langvad",
    "Solveig Schanke Eikum": "Solveig Schanke Eikum",
    "Sondre Myhre": "Sondre Moen Myhre",
    "Steffen S. Aagedal": "Steffen S. Aagedal",
    "Stella Heieren": "Stella Heieren",
    "Sten Ånnerud": "Sten J. Mcneil Ånnerud",
    "Stig Siversten": "Stig Siversten",
    "Stine G. Bergo": "Stine G. Bergo",
    "Studentparlamenett Uio": "Studentparlamentet Uio",
    "Stéphane Lelarge": "Stéphane Lelarge",
    "Sunniva R. Skjeggestad": "Sunniva R. Skjeggestad",
    "Tea Grav Rosenberg": "Thea Grav Rosenberg",
    "Thea Finstad": "Thea Nathalie Finstad",
    "Thea Urdal": "Thea Håmo Urdal",
    "Thomas A Syvertsen": "Thomas André Syvertsen",
    "Thorbjørn Kringlebotn Borlaug": "Thorbjørn Kringlebotn Borlaug",
    "Thorbjørn Kringlebotn": "Thorbjørn Kringlebotn",
    "Tia Karlsen": "Tia Kristine Karlsen",
    "Tone C. S. Thorgrimsen": "Tone Celine Sundseth Thorgrimsen",
    "Tonje T. Larsen": "Tonje T. Larsen",
    "Tonje Thielsen": "Tonje Thilesen",
    "Torbjørn S. Holen": "Torbjørn Sundal Holen",
    "Torgeir Mortensen": "Torgeir Mortensen",
    "Translation By Ingrid F. Brubaker": "Ingrid F. Brubaker",
    "Universitas´ Juleølpanel": "Universitas´ Juleølpanel",
    "Universitas’ Sommerøltestpanel": "Universitas’ Sommerøltestpanel",
    "Unni Claussen": "Unni Claussen",
    "Vegard Kristiansen Kvaale": "Vegard Kristiansen Kvaale",
    "Victor Andreas Shammas": "Victor Lund Shammas",
    "Vida Sundseth Brenna": "Vida Sundseth Brenna",
    "Vilde G. Larsen": "Vilde Grønn Larsen",
    "Vilde Sagstad Imeland": "Vilde Sagstad Imeland",
    "Wanda Nordstrøm": "Wanda Nathalie Nordstrøm",
    "Wenche Åsheim": "Wenche D. Åsheim",
    "Åshild Støylen": "Åshild Støylen",
    "Åsmund Øvsthus": "Åsmund P. Øvsthus",
    "Øistein Ø. Svelle": "Øistein Ø. Svelle",
    "Øyvind Østerås": "Øyvind Østerås",
}

In [196]:
cnn = Contributor.objects.extra(where=["CHAR_LENGTH(aliases) > 20"])
for cn in cnn:
    name = cn.display_name
    if name in fix:
        cn.display_name = fix[name]
        cn.aliases = ''
        print(f'{name:<30} -> {cn.display_name}')
        cn.save()

In [197]:
# Disse to personenen er forskjellige
{"Helena Nielsen": "Lena Nielsen"}
helena = Contributor.objects.get(display_name='Helena Nielsen')
lena = Contributor.objects.get_or_create(display_name='Lena Nielsen')[0]
helena.byline_set.filter(credit='photo').update(contributor=lena)

0

In [200]:
# remove useless contributors
from django.db.models import Count
allcn = Contributor.objects.order_by('display_name').annotate(bl_count=Count('byline', unique=True), stint_count=Count('stint', unique=True))
for cn in allcn.filter(bl_count=0):
    if not cn.email and cn.stint_count == 0:
        print(f'{str(cn.pk):<6} {str(cn):<40} {cn.bl_count} {cn.stint_count} {cn.user} {cn.email}')
        cn.delete()

In [194]:
# find possible misspelled names
from collections import Counter
cnn = Contributor.objects.extra(where=["CHAR_LENGTH(aliases) > 20"])
for cn in cnn:
    aliases = Counter(cn.aliases.strip().split('\n'))
    aliases = (f'"{k}" ({v}),' for k, v in aliases.items() if v > 1)
    print(f'"{cn}": {"|".join(aliases)}')

"Studentmållaget I Oslo": 
"Conflict Studies)": 
"Det Norske Studentersamfund": 
"NN": "Paul Patrick Borhaug" (2),
"Christina Bock Bjørn–Hansen": 
"Universitas’ Matpanel. Foto: Ketil Blom": 
"Akademikernes Internasjonale Hjelpefond)": 
"Melike Leblebicio?Lu Ketil Blom": 
